# Análisis de Palabras que no están en Diccionario

In [1]:
import enchant
import unicodecsv
import datetime
from apps import *
from nltk.tokenize import TweetTokenizer
import csv
tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
import pandas as pd
import json

d = enchant.Dict("es_AR")
path = 'train/train_' 

## Por provincia :
    - Cantidad de ocurrencia de cada palabra
    - Cantidad de usuarios de cada palabra

In [2]:
def dictionary(provincia,completo):
    dicc = {}
    dicc_usuarios = {}
    cant_words = 0
    file_path = path + provincia + '.csv'
    
    with open(file_path, 'r') as f:
        reader = unicodecsv.reader(f, encoding="utf-8", delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            #row[2] es la columna de texto
            if (completo):
                texto = [w for w in tknzr.tokenize(row[2]) if w.isalpha()]
            else:
                texto = [w for w in tknzr.tokenize(row[2]) if w.isalpha() and (not d.check(w))]
            
            
            uid = row[1]    # la columna de user id
            
            for w in texto:
                dicc[w] = 1 if not dicc.has_key(w) else dicc[w] +1
                if dicc_usuarios.has_key(w):
                    if uid not in dicc_usuarios[w]:
                        dicc_usuarios[w].append(uid)
                else:
                    dicc_usuarios[w] = [uid]
    return dicc,dicc_usuarios

def cant_palabras(dicc):
    return sum(dicc.values())

def save_dicts(pais,completo):
    words = {}
    cant_words = {}
    dicc_usuarios = {}
    for prov in pais:
        cant_words[prov] = 0
    for prov in pais:
        start = datetime.datetime.now()
        words[prov],dicc_usuarios[prov] = dictionary(prov,completo)
        cant_words[prov] = cant_palabras(words[prov])
        end = datetime.datetime.now()
        print prov,cant_words[prov], end - start
    return (words,cant_words,dicc_usuarios)


## Creo el un data frame con toda la data
    - Sumo las ocurrencias de cada palabra en todas las provincias.
    - Agrego las columnas de Frecuencias normalizadas (#palabras por millón) teniendo en cuenta también las palabras que se encontraban en el Diccionario de pyenchant

In [20]:
wcd = save_dicts(argentina,False)
wcd2 = save_dicts(argentina,True)

words2 = wcd2[0]
cant_words2 = wcd2[1]
dicc_usuarios2 = wcd2[2]

words = wcd[0]
cant_words = wcd[1]
dicc_usuarios = wcd[2]



cantPorProvincia = {prov:sum (words2[prov].values()) for prov in argentina}


users_cant = {}
for prov in argentina:
    for pal,lista in dicc_usuarios[prov].iteritems():
        #print prov,pal,lista
        if not users_cant.has_key(prov):
            users_cant[prov] = {}
            users_cant[prov][pal] = len(lista)
        else:
            users_cant[prov][pal] = len(lista)
df = pd.DataFrame(words)
df1 = pd.DataFrame(users_cant)
print df.shape
print df1.shape
df = df.fillna(0)
df1 = df1.fillna(0)
df.columns = [str(col) + 'Palabras' for col in df.columns]
df1.columns = [str(col) + 'Personas' for col in df.columns]
df1['cantUsuariosTotal'] = df1.sum(axis=1)
df['cantPalabra'] = df.sum(axis=1)

result = pd.concat([df, df1], axis=1)

for prov in argentina:
    result['fnorm_'+prov] = result[prov+'Palabras']/ cantPorProvincia[prov]



jujuy 129718 0:00:31.534116
catamarca 124611 0:00:31.765197
sanjuan 107572 0:00:28.750701
salta 125364 0:00:30.158193
rionegro 121512 0:00:31.018247
lapampa 126045 0:00:33.280261
chaco 121466 0:00:31.259404
mendoza 120822 0:00:32.098668
buenosaires 109740 0:00:29.517819
entrerios 127519 0:00:34.056412
chubut 117617 0:00:30.728476
santacruz 111277 0:00:24.959739
neuquen 123272 0:00:27.003130
misiones 117191 0:00:29.488951
corrientes 108869 0:00:27.531685
formosa 121008 0:00:29.613645
santafe 137300 0:00:31.947562
santiago 113373 0:00:28.036337
cordoba 114930 0:00:30.522226
larioja 127300 0:00:28.661648
tierradelfuego 118766 0:00:29.531143
tucuman 109161 0:00:27.259446
sanluis 127791 0:00:29.392325
jujuy 863853 0:00:11.451061
catamarca 868064 0:00:11.699863
sanjuan 821173 0:00:11.500546
salta 848813 0:00:10.772474
rionegro 887305 0:00:11.551112
lapampa 975122 0:00:12.753772
chaco 900357 0:00:12.442993
mendoza 917013 0:00:11.913522
buenosaires 846276 0:00:10.530784
entrerios 929788 0:00:1

### Me quedo con las palabras que fueron escritas por más de 5 personas distintas y ocurrieron más de 20 veces 

In [47]:
resultado = (result[(result.cantUsuariosTotal > 5 ) & (result.cantPalabra > 40 ) ]).copy()
#col = pd.Series(result.index)
#result['enDiccionario'] = map(lambda x: d.check(x),result.index)
resultado.shape

(6808, 71)

In [32]:

resultado.sort_values(['cantPalabra','cantUsuariosTotal'],ascending=[True,False]).head(100)


,buenosairesPalabras,catamarcaPalabras,chacoPalabras,chubutPalabras,cordobaPalabras,corrientesPalabras,entreriosPalabras,formosaPalabras,jujuyPalabras,lapampaPalabras,...,fnorm_misiones,fnorm_corrientes,fnorm_formosa,fnorm_santafe,fnorm_santiago,fnorm_cordoba,fnorm_larioja,fnorm_tierradelfuego,fnorm_tucuman,fnorm_sanluis
apurateee,3.0,10.0,6.0,7.0,2.0,8.0,5.0,5.0,1.0,2.0,...,0.000002,0.000009,0.000005,5.995360e-06,0.000001,0.000002,0.000000,0.000006,0.000001,0.000007
pelotudaaa,2.0,6.0,8.0,4.0,6.0,1.0,4.0,6.0,2.0,0.0,...,0.000003,0.000001,0.000007,7.993813e-06,0.000003,0.000006,0.000005,0.000005,0.000004,0.000009
merecia,2.0,8.0,4.0,7.0,4.0,3.0,5.0,5.0,6.0,2.0,...,0.000003,0.000004,0.000005,2.997680e-06,0.000001,0.000004,0.000005,0.000005,0.000004,0.000006
mantecol,7.0,6.0,6.0,7.0,1.0,3.0,6.0,3.0,1.0,6.0,...,0.000000,0.000004,0.000003,3.996906e-06,0.000002,0.000001,0.000001,0.000005,0.000005,0.000007
tiernooo,6.0,12.0,4.0,3.0,2.0,7.0,5.0,9.0,8.0,0.0,...,0.000003,0.000008,0.000010,2.997680e-06,0.000003,0.000002,0.000002,0.000005,0.000004,0.000006
mani,1.0,3.0,3.0,5.0,10.0,0.0,8.0,3.0,4.0,3.0,...,0.000002,0.000000,0.000003,4.996133e-06,0.000013,0.000010,0.000002,0.000008,0.000004,0.000009
perfeccion,4.0,5.0,4.0,3.0,3.0,6.0,0.0,6.0,11.0,1.0,...,0.000004,0.000007,0.000007,4.996133e-06,0.000009,0.000003,0.000003,0.000004,0.000002,0.000007
friday,1.0,1.0,3.0,2.0,6.0,11.0,0.0,4.0,7.0,2.0,...,0.000012,0.000013,0.000004,1.998453e-06,0.000002,0.000006,0.000005,0.000001,0.000004,0.000003
pagani,7.0,1.0,4.0,4.0,3.0,5.0,18.0,2.0,1.0,6.0,...,0.000002,0.000006,0.000002,2.098376e-05,0.000005,0.000003,0.000002,0.000003,0.000000,0.000002
changuito,4.0,19.0,32.0,3.0,0.0,0.0,0.0,0.0,3.0,1.0,...,0.000002,0.000000,0.000000,0.000000e+00,0.000023,0.000000,0.000006,0.000000,0.000005,0.000001


### Escribo el dataframe en palabras.csv 

In [48]:
resultado.sort_values(['cantPalabra','cantUsuariosTotal'],ascending=[False,False]).to_csv('palabras.csv',encoding='utf-8')